# Predictions for the MBTA #

## Motivation ##

TODO

## Methodology ##

### Setup ###

In [1]:
# Libraries.
import os
import itertools

import matplotlib, matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from sklearn import cross_validation, grid_search
from sklearn import linear_model

sns.set_style("whitegrid")
sns.set_context("paper")
% matplotlib inline

### Data Wrangling ###

#### MBTA ####

The MBTA provided us with entry data for each station at 15 minute intervals. After some wrangling, our base data set was composed of a ***stations*** data set and a ***gate count*** data set.

##### Stations #####

The station data set contained basic information for each station. One row corresponded to one station. Some of the data, in particular the latitude and longitude, was scraped from the web.

- ***stationid***: The unique identifier for the station.
- ***name***: The full name of the station station.
- ***line_1***: The primary line of the station (red/green/blue/orange).
- ***line_2***: The secondary line of the station (red/green/blue/orange). Only a few stations, such as park street (green/red line), had a value for this field.
- ***lat***: The latitude of the station.
- ***lon***: The longitude of the station.

In [3]:
stations = pd.read_csv('../../../data/stations.csv', low_memory=False)
stations.head()

,stationid,name,line_1,line_2,lat,lon
0,1002,Andrew Square,Red,NaN,42.329550,-71.056960
1,1004,JFK/U Mass,Red,NaN,42.321438,-71.052393
2,1005,North Quincy,Red,NaN,42.274816,-71.029176
3,1006,Wollaston,Red,NaN,42.265615,-71.019402
4,1007,Quincy Center,Red,NaN,42.250879,-71.004798


##### Gate Count #####

The gate count data set contained basic information for entries at each station. One row corresponded to the number of entries at a particular 15 minute interval for a station.

- ***locationid***: The unique identifier for the station.
- ***entries***: The number of entries for 15 minute interval.
- ***exits***: The number of exits for the 15 minute interval (NOT USED - exists are unreliable due to the nature of the system).
- ***service_day***: The actual day the service started (services on weekends can run into the next day).
- ***service_datetime***: The 15 minute interval where the entries/exists were aggregated.

In [3]:
gatecounts = pd.read_csv('../../../data/gatecounts.csv', low_memory=False)
gatecounts.head()

,locationid,entries,exits,service_day,service_datetime
0,1002,0,1,2013-01-01 00:00:00,2013-01-01 03:00:00
1,1002,1,0,2013-01-01 00:00:00,2013-01-01 05:00:00
2,1002,2,0,2013-01-01 00:00:00,2013-01-01 05:15:00
3,1002,3,0,2013-01-01 00:00:00,2013-01-01 05:30:00
4,1002,6,0,2013-01-01 00:00:00,2013-01-01 05:45:00


#### Weather ####

The weather data was obtained by scraping the [wunderground](http://www.wunderground.com/) API. Each row corresponded to the weather for a single day in Boston. A sample of the data is provided below. The most important feature was the "snow_fall" column, which was the recorded snow fall for that day.

In [5]:
weather = pd.read_csv('../../../data/weather.csv', low_memory=False)
weather.head()

,date,fog,hail,rain,snow,temp_min,temp_max,temp_mean,rain_fall,snow_fall,wind_speed,vis_min,vis_max,vis_mean
0,2012-01-01 00:00:00,0,0,1,0,39,52,46,0.01,0,9,9,10,10
1,2012-01-02 00:00:00,0,0,1,0,34,50,42,0.01,0,14,10,10,10
2,2012-01-03 00:00:00,0,0,0,0,14,35,25,0.00,0,15,10,10,10
3,2012-01-04 00:00:00,0,0,0,0,10,28,19,0.00,0,12,10,10,10
4,2012-01-05 00:00:00,0,0,0,0,25,39,32,0.00,0,12,10,10,10


#### Game Days ####

The game day data for Red Sox games was scraped directly from their website while the game day data for the Celtics and Bruines game was scraped from ESPN. Each row corresponded to a day in which a game (or games) occurred. A sample of the data is provided below. The most important features for daily prediction purposes were:

- ***bruins_game***: Binary column representing if a Bruins game occurred on this day.
- ***celtics_game***: Binary column representing if a Celtics game occurred on this day.
- ***sox_game***: Binary column representing if a Red Sox game occurred on this day.

In [4]:
gamedays = pd.read_csv('../../../data/game_days.csv', low_memory=False)
gamedays.head()

,date,weekday,bruins_playoff,bruins_opponent,bruins_time,celtics_playoff,celtics_opponent,celtics_time,sox_time,sox_opponent,...,sox_sat_early,sox_sat_late,sox_sun_early,sox_sun_late,sox_mon_thu_early,sox_mon_thu_late,sox_mon_fri_early,sox_mon_fri_late,sox_fri_sat_early,sox_fri_sat_late
0,2012-04-13,4,NaN,NaN,NaN,NaN,NaN,NaN,14:05:00,Rays,...,0,0,0,0,0,0,1,0,1,0
1,2012-04-14,5,NaN,NaN,NaN,NaN,NaN,NaN,16:05:00,Rays,...,1,0,0,0,0,0,0,0,1,0
2,2012-04-15,6,NaN,NaN,NaN,NaN,NaN,NaN,13:35:00,Rays,...,0,0,1,0,0,0,0,0,0,0
3,2012-04-16,0,NaN,NaN,NaN,NaN,NaN,NaN,11:05:00,Rays,...,0,0,0,0,1,0,1,0,0,0
4,2012-04-17,1,NaN,NaN,NaN,NaN,NaN,NaN,19:10:00,Rangers,...,0,0,0,0,0,1,0,1,0,0


#### Daily Data ####

The daily data set was an aggregation of entries per day for each station, with a few additional features. Sparing the details on how we generated these additional features (please refer to the ***features*** section of our repository), some important ones to recognize are:

- ***entries_weeks_ago_1***: The number of entries for this station on the same day one week ago.
- ***snow_fall***: The amount of snow the fell for that day, in inches.
- ***snow_accum***: The amount of snow accumulated up to the current day. The snow accumulation was calculated using a quasi-linear decay function based on the snow fall of the previous two weeks.
- ***dist_to_center***: The distance (in kilometers) to the center of the city (city hall).

#### Daily Data w/o Holidays ####

After some thorough analysis, we decided that "holidays" had no consistent pattern. We decided to consider them "extreme outliers" and removed them from our base data set (future analysis would be necessary for holidays). The "holidays" we removed were:

- New Years Eve
- New Years Day
- Boston Marathon
- Fourth of July
- Boston Move Out Day (September 1st)
- Thanksgiving
- Black Friday
- Christmas Week (12/25, 12/26, 12/27).

#### Daily Data w/o Weekends ####

After some thorough analysis, we decided that weekends and weekdays had two completely seperate patterns. We wanted to assist the MBTA in the greatest way possible and concentrated on weekdays, removing weekends from our base data set (future analysis would be necessary for weekends).

#### Final Data ####

Our final data set consisted of daily entries for each station, exluding holidays and weekends, with the additional weather and game features.

In [2]:
data = pd.read_csv('../../../data/mbta_daily_weather_games_noholidays_weekdays.csv', low_memory=False)
print("Rows: " + str(data.shape[0]))
print("Cols: " + str(data.shape[1]))
data = data.reset_index(drop=True)
data.head()

Rows: 32547
Cols: 50


,locationid,service_day,entries,name,line_1,line_2,lat,lon,service_datetime,fog,...,snow_fall_predict,snow_accum,snow_accum_predict,dist_to_center,entries_weeks_ago_1,entries_weeks_ago_2,entries_weeks_ago_3,bruins_game,celtics_game,sox_game
0,1002,2013-01-03 00:00:00,5733,Andrew Square,Red,NaN,42.329550,-71.056960,2013-01-03 05:00:00,0,...,0,0,0,3.404767,NaN,NaN,NaN,0,0,0
1,1004,2013-01-03 00:00:00,5842,JFK/U Mass,Red,NaN,42.321438,-71.052393,2013-01-03 04:45:00,0,...,0,0,0,4.328881,NaN,NaN,NaN,0,0,0
2,1005,2013-01-03 00:00:00,6139,North Quincy,Red,NaN,42.274816,-71.029176,2013-01-03 03:15:00,0,...,0,0,0,9.777437,NaN,NaN,NaN,0,0,0
3,1006,2013-01-03 00:00:00,4028,Wollaston,Red,NaN,42.265615,-71.019402,2013-01-03 05:00:00,0,...,0,0,0,10.976943,NaN,NaN,NaN,0,0,0
4,1007,2013-01-03 00:00:00,7331,Quincy Center,Red,NaN,42.250879,-71.004798,2013-01-03 04:30:00,0,...,0,0,0,12.909591,NaN,NaN,NaN,0,0,0


##  Prediction ##

### Introduction ###

TODO: FIX

Changes in the accuracy of entry predictions are one way of determining the importance of snow to the MBTA. To do so, we can add snow "features" (or predictors) to a basic model (Linear Regression) and check for a reduction in the overall percent error of our predictions for each station. We can define percent error for each station as:

$$ station_{percent\ error} = \frac{|entries_{predicted} - entries_{actual}|}{entries_{actual}} * 100 $$

Once we have determined the percent error for each station, we can determine the overall average of errors:

$$ Total\ Average\ Error = \frac{1}{n} \sum^n_i{station^i_{percent\ error}} $$

### Method Definitions ###

The following methods are used to help us generate a prediction model and determine the percent error for each station.

In [5]:
"""
Generates a prediction model for a given station with desired features.

This will return the best model based on 10 fold cross-validation with an 70/30 train/test split.

In addition, this will return the mean absolute error for all the entries in the test set, and any outliers
based on a given absolute error threshold.

Key arguments:

  station    -- The station to train on.
  cols       -- The columns in the station data frame that are considered features.
  predictor  -- The predictor to use (sklearn predictor)
  parameters -- The parameters for the predictor.
  rstate     -- The random state to use for the train/test split.

"""
def predict(station, features, predictor, parameters, rstate):
    # Copy the station so we don't manipulate the original.
    station = station.copy()
    
    # Get the columns of the dataframe.
    all_cols = list(station.columns)
    
    # Determine the indices of the columns.
    y_col_indices = [0] + list(np.where([col == 'entries' for col in all_cols])[0] + 1)
    x_col_indices = [0] + list(np.where([col in features for col in all_cols])[0] + 1)
    
    # Make sure none of the predictor fields are null.
    for col in features:
        station = station[pd.notnull(station[col])]
    
    # Remove any entries where no one was there (probably closed) or the entries appears to be
    #  extremely low (less than 1% of the mean).
    # We assume that any entries with less than 1% of the normal entries is having serious issues.
    min_entries = np.mean(station['entries']) * .005
    station = station[station['entries'] > min_entries]
    
    # Reset the station indices, we have to reset twice so the matrix values gets the index column.
    station = station.reset_index()
    station.drop('index', axis=1, inplace=True)
    station = station.reset_index()
    
    # Get the dataframe as a matrix where the first column is the index.
    matrix = station.values
    
    # Slice so the y only contains 2 column (index, entries)
    #  and the x is a matrix that contains the index and all the predictors.
    y = matrix[:,y_col_indices]
    x = matrix[:,x_col_indices]
    
    # Split the data set into a train and test.
    x_train, x_test, y_train, y_test = cross_validation.train_test_split(x, y, test_size=0.3, random_state=rstate)
    
    # Convert the train and test sets into a format sklean fit() expects.
    x_train_fit = np.matrix(x_train[:,1:], dtype=np.float32)
    y_train_fit = np.array([v[0] for v in y_train[:,1:]], dtype=np.uint16)
    
    x_test_fit = np.matrix(x_test[:,1:], dtype=np.float32)
    y_test_fit = np.array([v[0] for v in y_test[:,1:]], dtype=np.uint16)
    
    # Train using a grid search based on the parameters provided.
    clf = grid_search.GridSearchCV(predictor, parameters, scoring='mean_squared_error', cv=10)
    clf.fit(x_train_fit, y_train_fit)
    
    # Determine what the best model was.
    model = clf.best_estimator_
    
    # Find the train/test data sets in the original station data frame.
    train_results = station.iloc[y_train[:,0]]
    test_results = station.iloc[y_test[:,0]]
    
    # Predict using the train/test set.
    train_results['prediction'] = model.predict(x_train_fit)
    test_results['prediction'] = model.predict(x_test_fit)
    
    # Determine the percent errors.
    train_results['error'] = np.abs(train_results['prediction'] - train_results['entries']) / train_results['entries']
    test_results['error'] = np.abs(test_results['prediction'] - test_results['entries']) / test_results['entries']
    
    # Determine overall error for station for training and test.
    train_error =  np.sum(np.abs(train_results['prediction'] - train_results['entries'])) / np.sum(train_results['entries'])
    test_error =  np.sum(np.abs(test_results['prediction'] - test_results['entries'])) / np.sum(test_results['entries'])
    
    return model, train_error, test_error, train_results, test_results

In [7]:
"""
Tests a station multiple times by running the prediction method with a different
random state.

Key arguments:

  station  -- The station to test for.
  cols     -- The columns in station to use as features.
  predictor  -- The predictor to use (sklearn predictor)
  parameters -- The parameters for the predictor.
  trials   -- The number of times to run predict (also the number of randomly generated test sets).

"""
def test_features(station, cols, predictor, parameters, error_threshold = 0.2, trials = 50):
    train_errors = np.zeros(trials)
    test_errors = np.zeros(trials)
    train_outliers = pd.DataFrame()
    test_outliers = pd.DataFrame()
    
    for rstate in xrange(trials):
        model, train_error, test_error, train_results, test_results = predict(station, cols, predictor, parameters, rstate)
        
        # Keep track of the train and test errors for this result.
        train_errors[rstate] = train_error
        test_errors[rstate] = test_error
        
        # Keep track of any "outliers" for both the train and test group, i.e. rows that were above the threshold.
        trial_train_outliers = train_results[train_results['error'] > error_threshold]
        trial_test_outliers = test_results[test_results['error'] > error_threshold]
        
        if len(train_outliers) == 0 and len(test_outliers) == 0:
            train_outliers = trial_train_outliers
            test_outliers = trial_test_outliers
        else:
            train_outliers = pd.concat([train_outliers, trial_train_outliers[~trial_train_outliers.isin(train_outliers.index)]])
            test_outliers = pd.concat([test_outliers, trial_test_outliers[~trial_test_outliers.isin(test_outliers.index)]])
    
    return train_errors, test_errors, train_outliers, test_outliers

### A Simple Example ###

Lets step through a simple example of how the process works. For this simple example, we're going to predict the entries for Andrew Square.

In [11]:
andrew_square = data[data['locationid'] == 1002]
predictor = linear_model.LinearRegression()
parameters = {}

For a group of days (test set), we want to see how accruate we can predict the number of daily entries for Andrew Square using only the number of entries on the same day for the past two weeks.

E.g. If I was predicting the entries for Andrew Square on 2015-02-21, I would build a regression model only using the known entries on 2015-02-14 and 2015-02-07 as features.

The method below builds a model and predicts the entries for multiple test sets. We can than average the error for each test set to determine the mean prediction error for Andrew Square.

In [15]:
train_errors, test_errors, train_outliers, test_outliers = test_features(andrew_square, ['entries_weeks_ago_1', 'entries_weeks_ago_2'], predictor, parameters, .2, 50)
print("Average Percent Error for Test Sets: " + str(np.mean(test_errors)))

Average Percent Error for Test Sets: 0.061955987446


> _We can predict entries at Andrew Square with an 6% error rate._

We can also take a look at days in which the prediction model did not perform well (> 20% error).

In [16]:
test_outliers.head()

,index,locationid,service_day,entries,name,line_1,line_2,lat,lon,service_datetime,...,snow_accum_predict,dist_to_center,entries_weeks_ago_1,entries_weeks_ago_2,entries_weeks_ago_3,bruins_game,celtics_game,sox_game,prediction,error
336,336,1002,2014-05-26 00:00:00,2809,Andrew Square,Red,NaN,42.32955,-71.05696,2014-05-26 05:00:00,...,0,3.404767,6038,6215,6231,0,0,0,6161.294728,1.193412
75,75,1002,2013-05-03 00:00:00,6774,Andrew Square,Red,NaN,42.32955,-71.05696,2013-05-03 03:15:00,...,0,3.404767,6549,312,6186,0,0,0,4626.313037,0.317049
434,434,1002,2014-10-13 00:00:00,4272,Andrew Square,Red,NaN,42.32955,-71.05696,2014-10-13 05:00:00,...,0,3.404767,6716,6288,6321,1,0,0,6275.618417,0.469012
188,188,1002,2013-10-14 00:00:00,4212,Andrew Square,Red,NaN,42.32955,-71.05696,2013-10-14 04:30:00,...,0,3.404767,6385,6312,6392,1,0,0,6236.032889,0.480540
65,65,1002,2013-04-19 00:00:00,312,Andrew Square,Red,NaN,42.32955,-71.05696,2013-04-19 04:45:00,...,0,3.404767,6186,6947,6324,0,0,0,6381.087873,19.452205


### Finding the Best Feature Set ###

Determining features is generally the first step of the model generation process. For our scenario, we wanted to determine the feature set that results in the lowest possible average percent error across all stations. To accomplish this, we executed an exhaustive search of feature sets to determine which performed the best.

In [32]:
"""
Tests all stations with a set of features.

Key arguments:

  features -- The features to test.

"""
def test_features_for_all_stations(features):
    results = pd.DataFrame()
    
    predictor = linear_model.LinearRegression()
    parameters = {}
    
    # Test the feature sets for every single station.
    for locationid in np.unique(data['locationid']):
        result = pd.Series()
        
        # Get a particular station.
        station = data[data['locationid'] == locationid]
        
        # Add identifier information.
        result['locationid'] = locationid
        result['name'] = station['name'].iloc[0]
        
        # Get the mean percent error for the train/test set using the base feature.
        train_errors, test_errors, _, _ = test_features(station, features, predictor, parameters)
        result['train_error'] = np.mean(train_errors)
        result['test_error'] = np.mean(test_errors)
        result['features'] = str(features)
        
        # Save the result.
        results = results.append(result, ignore_index=True)
    
    return results

#### Base Feature Set ####

This feature set was included in all feature combinations (consider it the baseline feature set).

In [42]:
base_features = ['entries_weeks_ago_1']

#### Additional Feature Set ####

The "additional" features we wanted to test included weather, sports, and other aspects of the data.

In [43]:
other_features = [
    'entries_weeks_ago_2', 
    'entries_weeks_ago_3',
    ['day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4'],
    ['month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'],
    ['temp_min', 'temp_max', 'temp_mean'],
    ['rain', 'rain_fall'],
    ['snow', 'snow_fall', 'snow_accum'],
    ['bruins_game', 'celtics_game', 'sox_game'],
]

#### Exhaustive Search ####

For all possible combinations of "other features", we determined the average percent error across all stations.

In [44]:
results = pd.DataFrame()

i = 0
# Try every combination of the "other features".
for l in range(0, len(other_features) + 1):
    for subset in itertools.combinations(other_features, l):
        # Add on the base line features.
        features = list(base_features)
        # Build the feature list.
        for feature in subset:
            features += [feature] if not isinstance(feature, list) else feature
        
        # Save results to disk, since this takes a while and we don't want to lose results.
        results = results.append(test_features_for_all_stations(features), ignore_index=True)
        results.to_csv("../../results/prediction/feature_set_" + str(i) + ".csv")
        i += 1

In [45]:
featureset_results = pd.read_csv("../../results/prediction/feature_set_eval.csv", low_memory=False)
featureset_results.head()

,Unnamed: 0,features,locationid,name,test_error,train_error
0,0,['entries_weeks_ago_1'],1002,Andrew Square,0.062664,0.062752
1,1,['entries_weeks_ago_1'],1004,JFK/U Mass,0.142301,0.144566
2,2,['entries_weeks_ago_1'],1005,North Quincy,0.067818,0.069260
3,3,['entries_weeks_ago_1'],1006,Wollaston,0.061919,0.062765
4,4,['entries_weeks_ago_1'],1007,Quincy Center,0.070770,0.073322


In [46]:
grouped = featureset_results.groupby('features').mean().sort('test_error')
grouped.head(n=100)

,Unnamed: 0,locationid,test_error,train_error
features,,,,
"['entries_weeks_ago_1', 'entries_weeks_ago_2', 'entries_weeks_ago_3', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'rain', 'rain_fall', 'snow', 'snow_fall', 'snow_accum', 'bruins_game', 'celtics_game', 'sox_game']",15781,1068.126984,0.073578,0.066702
"['entries_weeks_ago_1', 'entries_weeks_ago_2', 'entries_weeks_ago_3', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'temp_min', 'temp_max', 'temp_mean', 'rain', 'rain_fall', 'snow', 'snow_fall', 'snow_accum', 'bruins_game', 'celtics_game', 'sox_game']",16096,1068.126984,0.073932,0.066307
"['entries_weeks_ago_1', 'entries_weeks_ago_3', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'rain', 'rain_fall', 'snow', 'snow_fall', 'snow_accum', 'bruins_game', 'celtics_game', 'sox_game']",15340,1068.126984,0.074003,0.067463
"['entries_weeks_ago_1', 'entries_weeks_ago_3', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'temp_min', 'temp_max', 'temp_mean', 'rain', 'rain_fall', 'snow', 'snow_fall', 'snow_accum', 'bruins_game', 'celtics_game', 'sox_game']",16033,1068.126984,0.074385,0.067094
"['entries_weeks_ago_1', 'entries_weeks_ago_2', 'entries_weeks_ago_3', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'snow', 'snow_fall', 'snow_accum', 'bruins_game', 'celtics_game', 'sox_game']",14143,1068.126984,0.074714,0.068212
"['entries_weeks_ago_1', 'entries_weeks_ago_2', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'rain', 'rain_fall', 'snow', 'snow_fall', 'snow_accum', 'bruins_game', 'celtics_game', 'sox_game']",14962,1068.126984,0.074897,0.068414
"['entries_weeks_ago_1', 'entries_weeks_ago_3', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'snow', 'snow_fall', 'snow_accum', 'bruins_game', 'celtics_game', 'sox_game']",12820,1068.126984,0.075117,0.068947
"['entries_weeks_ago_1', 'entries_weeks_ago_2', 'entries_weeks_ago_3', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'temp_min', 'temp_max', 'temp_mean', 'snow', 'snow_fall', 'snow_accum', 'bruins_game', 'celtics_game', 'sox_game']",15718,1068.126984,0.075144,0.067909
"['entries_weeks_ago_1', 'entries_weeks_ago_2', 'entries_weeks_ago_3', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'rain', 'rain_fall', 'snow', 'snow_fall', 'snow_accum', 'bruins_game', 'celtics_game', 'sox_game']",14647,1068.126984,0.075147,0.068834


In [69]:
top_features = ['entries_weeks_ago_1', 'entries_weeks_ago_2', 'entries_weeks_ago_3', 'rain', 'rain_fall', 'snow', 'snow_fall', 'snow_accum', 'bruins_game', 'celtics_game', 'sox_game']
top_group = featureset_results[featureset_results.features == str(top_features)]
print(top_group['test_error'].mean())
top_group.head()

0.0801623409944


,Unnamed: 0,features,locationid,name,test_error,train_error
11466,11466,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1002,Andrew Square,0.056496,0.053390
11467,11467,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1004,JFK/U Mass,0.132855,0.130215
11468,11468,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1005,North Quincy,0.060145,0.056720
11469,11469,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1006,Wollaston,0.052835,0.050158
11470,11470,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1007,Quincy Center,0.063423,0.060990


In [70]:
top_group[top_group.test_error > .1].head(n=20)

,Unnamed: 0,features,locationid,name,test_error,train_error
11467,11467,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1004,JFK/U Mass,0.132855,0.130215
11477,11477,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1015,Suffolk Downs,0.134443,0.124854
11495,11495,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1052,Park Street,0.102755,0.099663
11503,11503,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1060,Science Park,0.171557,0.165019
11509,11509,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1074,Community College,0.202653,0.193352
11517,11517,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1082,Ruggles,0.108381,0.103510
11518,11518,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1083,Roxbury Crossing,0.101898,0.096164
11523,11523,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1101,Riverside,0.112565,0.107576
11526,11526,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1115,Courthouse,0.130376,0.125163
11527,11527,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1116,World Trade Center,0.176576,0.163942


In [71]:
top_features2 = ['entries_weeks_ago_1', 'entries_weeks_ago_2', 'entries_weeks_ago_3', 'day_of_week_0', 'day_of_week_1', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4', 'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12', 'rain', 'rain_fall', 'snow', 'snow_fall', 'snow_accum', 'bruins_game', 'celtics_game', 'sox_game']
top_group2 = featureset_results[featureset_results.features == str(top_features2)]
print(top_group2['test_error'].mean())
top_group2.head()

0.0735780541681


,Unnamed: 0,features,locationid,name,test_error,train_error
15750,15750,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1002,Andrew Square,0.047974,0.043848
15751,15751,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1004,JFK/U Mass,0.096950,0.089662
15752,15752,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1005,North Quincy,0.054719,0.049506
15753,15753,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1006,Wollaston,0.048557,0.043757
15754,15754,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1007,Quincy Center,0.054422,0.049937


In [73]:
top_group2[top_group2.test_error > .1].head(n=20)

,Unnamed: 0,features,locationid,name,test_error,train_error
15761,15761,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1015,Suffolk Downs,0.145303,0.130427
15779,15779,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1052,Park Street,0.101463,0.093231
15787,15787,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1060,Science Park,0.163305,0.149041
15793,15793,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1074,Community College,0.159087,0.145437
15807,15807,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1101,Riverside,0.108393,0.099805
15810,15810,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1115,Courthouse,0.122374,0.112346
15811,15811,"['entries_weeks_ago_1', 'entries_weeks_ago_2',...",1116,World Trade Center,0.170949,0.152333


##### Conclusion #####

We found the best feature set to be.

## Conclusion ##

- *What are the most important features?*

- *What was the best model?*

- *What is the average prediction ratio?*

- *What are some known outliers?*